# **Representational similarity analysis: a tutorial with rsatoolbox**
written for [Diedrichsen Lab](https://www.diedrichsenlab.org) and [Brain and Mind Institute](https://www.uwo.ca/bmi/)

This tutorial will walk you through the basic concept to understand representational similarity analysis (RSA) and how to apply it using the rsatoolbox. [Click here](https://rsatoolbox.readthedocs.io/en/stable/) for more documentation about the rsatoolbox. To find out more about how RSA works and to learn how to use the PcmPy toolbox to perform RSA, you can check [this tutorial](https://github.com/mnlmrc/didactic/blob/main/rsa_pcmpy.ipynb).

In [10]:
import pickle
import rsatoolbox as rsa
import numpy as np

Example dataset
--
Throughout the tutorial, we use example fMRI data from an experiment in which participants responded to tactile stimuli on either the index or ring finger, following a visual cue that indicated the probability of stimulation for each finger. The visual cues were 0%-100% ring-index, 25%-75%, 50%-50%, 75%-25% or 100%-0%. To simplify, we will use condition labels only reporting the index probability, i.e., 0%, 25%, 50%, 75%, 100%. The data were collected in ten blocks. In the first-level GLM, we modelled neural activation $y_i$ in the $ i^{th} $ voxel as a linear combination of 5 regressors of interests, each corresponding to a probability cue:

$$ y_i = X \beta_i + \epsilon $$

The $\beta$ coefficients from each regressor indicate how much each voxel was activated for each probability cue. The goal of RSA is to understand whether voxels show different **activity patterns** between conditions, regardless of the overall amount of activation. Let's first load the data:

In [11]:
f = open('data_demo_smp.p', 'rb')
Y, ResMS, residuals, cond_vec, part_vec, cond_names = pickle.load(f)

To use the rsatoolbox, we first need to organize or data in a Dataset object. We want to compute the cross-validated [Mahalanobis distance](https://github.com/mnlmrc/didactic/blob/main/rsa_pcmpy.ipynb#Prewhitening-and-Mahalanobis-distance) between pairs of conditions, so before creating the dataset, we will prewhiten our data using the ResMS from the first-level GLM (i.e., univariate prewhitening). The `obs_descriptors` dictionary should contain the field `'conds'` corresponding to a vector of condition labels and the field `'run'` with partition labels (e.g., fMRI runs) to perform cross-validation.

In [12]:
dataset = rsa.data.Dataset(
        Y,
        channel_descriptors={
            'channel': np.array(['vox_' + str(x) for x in range(Y.shape[-1])])},
        obs_descriptors={'conds': cond_vec,
                         'run': part_vec},
    )

We can now use the function `calc_rdm` to compute the distances:

In [13]:
rdm = rsa.rdm.calc_rdm(dataset, method='crossnobis', descriptor='conds', cv_descriptor='run')

/Users/mnlmrc/Documents/GitHub/didactic/venv/lib/python3.13/site-packages/numpy/_core/numeric.py:442: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


The variable `rdm` contains a representation dissimilarity matrix, i.e., a matrix of squared cross-validated distances between conditions. For more information about RDMs objects and how to manipulate them, see [this documentation](https://rsatoolbox.readthedocs.io/en/stable/operations.html). Representation dissimilarity matrices stored in RDMs object can be directly plotted using the vis package in rsatoolbox:

In [14]:
rsa.vis.show_rdm(rdm, cmap='viridis')

(<Figure size 200x200 with 1 Axes>,
 array([[<Axes: >]], dtype=object),
 {-1: {}, 0: {'image': <matplotlib.image.AxesImage at 0x17c1956d0>}})